# Usage

## Dependencies
This project depends on working installations of:
* Biopython
* joblib
* MOODS
* numpy
* pandas
* pybedtools
* scikit-learn
* statsmodels
* tqdm

All dependencies are available via conda.

This notebook additionally depends on:
* wget
* zcat

### MOODS Motif scanning
Motif scanning functionality depends on the [MOODS Python module](https://github.com/jhkorhonen/MOODS/tree/master/python), which is available via [conda](https://anaconda.org/bioconda/moods). This may have to be installed manually if you don't use conda, since there is currently no PyPi package.


In [1]:
import pandas as pd
import numpy as np

import datetime
from timeit import default_timer as timer


In [2]:
sns.set(rc={'figure.figsize':(12,12)})

## Specify number of processes

In [31]:
n_jobs = 20

## Generate scored and stratified fasta file from peak data
For each peak, output should appear as:
```
>sequence_name score stratum other_descriptive_text
SEQUENCESEQUENCESEQUENCE
```

In this example, we set `score` to `log2fc * (1-pval)` using values from our peak file.
If strata are unimportant, you may simply enter a single constant.
If you already have such a file, skip to **Load data for motif enrichment**

### Download and locate reference sequence

In [3]:
! wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_28/GRCh38.primary_assembly.genome.fa.gz -O genome.fa.gz
! zcat genome.fa.gz > genome.fa

--2018-11-28 11:08:24--  ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_28/GRCh38.primary_assembly.genome.fa.gz
           => 'genome.fa.gz'
Resolving ftp.ebi.ac.uk... 193.62.192.4
Connecting to ftp.ebi.ac.uk|193.62.192.4|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/gencode/Gencode_human/release_28 ... done.
==> SIZE GRCh38.primary_assembly.genome.fa.gz ... 844691642
==> PASV ... done.    ==> RETR GRCh38.primary_assembly.genome.fa.gz ... done.
Length: 844691642 (806M) (unauthoritative)

GRCh38.primary_asse 100%[===================>] 805.56M  6.94MB/s    in 2m 57s  

2018-11-28 11:11:22 (4.56 MB/s) - 'genome.fa.gz' saved [844691642]



In [4]:
genome_fa_filename = f'genome.fa'

### Locate peak data

In [5]:
peaks_filename = 'differential_peaks.txt'

In [6]:
peaks_df = pd.read_table(peaks_filename)

### Annotate peaks with additional information

In [7]:
min_tag_limit = 10

peak_annotation_columns = ['chr', 'start', 'end', 'strand', 'log2fc', 'pval', 'min_tags']
peak_id_column = 'PeakID'

peak_annotation_df = (peaks_df[peaks_df['min_tags'] >= min_tag_limit]
                      .rename(columns = {peak_id_column: 'peak_id'})
                      [list(set(['peak_id'] + peak_annotation_columns))]
                      .rename(columns = {col: f'peak_{col}' 
                                         for col 
                                         in peak_annotation_columns})
                      .drop_duplicates())

peak_annotation_df = peak_annotation_df[['peak_id'] + [f'peak_{col}' for col in peak_annotation_columns]]


### Weight log2fc by 1-pval

In [8]:
peak_annotation_df['peak_weighted_log2fc'] = peak_annotation_df['peak_log2fc'] * (1 - peak_annotation_df['peak_pval'])

### Set score for each peak equal to the weighted log2fc

In [9]:
peak_annotation_df['peak_score'] = peak_annotation_df['peak_weighted_log2fc']
peak_annotation_bed_columns = ['peak_chr', 'peak_start', 'peak_end', 'peak_id', 'peak_score', 'peak_strand']

In [10]:
peak_annotation_df[peak_annotation_bed_columns].head()

,peak_chr,peak_start,peak_end,peak_id,peak_score,peak_strand
0,chr15,65295976,65296126,chr15-1,1.469470,+
1,chr6,26032024,26032174,chr6-1,0.220398,-
2,chr1,44721828,44721978,chr1-1,1.132725,-
3,chr1,153990688,153990838,chr1-2,0.998365,+
4,chr12,6943742,6943892,chr12-1,1.689156,+


### Extract peak sequence +/- 300 from peak center

In [11]:
sequence_length = 600

In [12]:
peak_annotation_df.head()

,peak_id,peak_chr,peak_start,peak_end,peak_strand,peak_log2fc,peak_pval,peak_min_tags,peak_weighted_log2fc,peak_score
0,chr15-1,chr15,65295976,65296126,+,1.469470,0.000000e+00,23865.0,1.469470,1.469470
1,chr6-1,chr6,26032024,26032174,-,0.220398,1.330000e-107,18738.0,0.220398,0.220398
2,chr1-1,chr1,44721828,44721978,-,1.132725,0.000000e+00,9718.0,1.132725,1.132725
3,chr1-2,chr1,153990688,153990838,+,0.998365,0.000000e+00,8916.0,0.998365,0.998365
4,chr12-1,chr12,6943742,6943892,+,1.689156,0.000000e+00,5875.0,1.689156,1.689156


In [13]:
peak_annotation_bed_columns

['peak_chr', 'peak_start', 'peak_end', 'peak_id', 'peak_score', 'peak_strand']

### Write fasta file

In [14]:
from meirlop import get_centered_peak_sequences, get_gc_pct, get_gc_pct_bin, write_scored_fasta
peak_fasta_filename = 'peak_scores.fa'
peak_fasta_file = open(peak_fasta_filename, 'w')

peak_sequence_dict, peak_sequence_bed_df = get_centered_peak_sequences(peak_annotation_df, 
                                                                       genome_fa_file = open(genome_fa_filename, 'r'), 
                                                                       sequence_length = sequence_length, 
                                                                       peak_bed_columns = peak_annotation_bed_columns)
peak_score_dict = peak_annotation_df.set_index('peak_id')['peak_score'].to_dict()

peak_gc_pct_dict = {peak_id: get_gc_pct(seq) 
                        for peak_id, seq 
                        in peak_sequence_dict.items()}

peak_gc_pct_bin_dict = {peak_id: get_gc_pct_bin(seq) 
                        for peak_id, seq 
                        in peak_sequence_dict.items()}

peak_fasta_string = write_scored_fasta(peak_sequence_dict, 
                                       peak_score_dict, 
                                       peak_fasta_file, 
                                       other_dicts = [peak_gc_pct_bin_dict])
peak_fasta_file.close()

***** WARNING: File /tmp/pybedtools.j0f4dze7.tmp has inconsistent naming convention for record:
GL000008.2	0	209709

***** WARNING: File /tmp/pybedtools.j0f4dze7.tmp has inconsistent naming convention for record:
GL000008.2	0	209709

index file genome.fa.fai not found, generating...


In [15]:
! head {peak_fasta_filename}

>chr9-2 10.335965736376739 35
TTCTAAATAACTTACAAAGATGTGTCAAGAAGCAATAACAAGACATATTAGAAAATATTTTGAATGGAATGAATATAAAAACCTGCCAAAACAGAACATGTGTGATGTAATTAAAGCTTGAAATTCTCACAGGACAAAATAAGAAGCATCTAATATCGATGTTCTTGGATTCCTCCTTTGCTAATGGCTGATTAGCTCTTATTGAACCAATTTTTACACAGATAATTGTAAACAACAGGAGATACTCTAAAACAAACAAACAAACAAACAAAAAACTATTTGAAGTTATCAGAGACTGACCAAAAGCAGGTAGATGTTGAAGGTGACAACACTTTCTAGAAAGGAACAGTACTGAGTAGGCACCCTGGTTTCACCACTTTTAGCCTGAAGGTAGATCCTGATGCAGCTAAAATATGGAAGTGAGGAAACTTTCTTATTGGAGTGATGAACCAGAGGGCAGAGGCAGAATATTTTACAATCAGAAGAAGGGTAGGGTTGTGAGGCTATCTCCAAAATGAGAGATAAGGTAGAAACACTATGTTATACCTAAATATCTGGCTTAAGTCTAACCAAAATACATGGATAAGACTACAAACAAAAG
>chr1-234 4.925634316875965 35
GTTAACTATTAGCTTTACTGCAGTTTCTTCTTGGTAGTTATCATTTCATGCCTGCCTACATACATACAAGGGGACCAGTGATAGTTTTTATGTGCTCAGCAAGATTTTTTTTTTCTTTCTTTCTCTTCCTAGTGAGGAAAAAGAAAGTTAGTGGCAGTTGGCATGCTGCCAGCTGAGTTTTTTTGCTGCTTTGAGTTTCAGGTTTCTTTCTTTCTCTTCCTAGTGAGGAAAAAGAAAGTTAGTGGCAGTTGGCATGCTGCCAGCTGAGTTTTTTTGCTGCTTTGAGTCTCAGTTTTCTTTCTTTCCTAGAGTCTCTGAAGCCACAGATCTCTTAAGA

## Load data for motif enrichment

### Load the scored fasta

In [16]:
from meirlop import read_scored_fasta, dict_to_df
sequence_dict, score_dict, description_dict = read_scored_fasta(open(peak_fasta_filename, 'r'), description_delim = ' ')
strata_dict = {key: int(val[2]) for key, val in description_dict.items()}

score_df = dict_to_df(score_dict, 'peak_id', 'peak_score')
strata_df = dict_to_df(strata_dict, 'peak_id', 'peak_strata')

In [17]:
print(score_df.shape)
score_df.head()

(30474, 2)


,peak_id,peak_score
0,chr9-2,10.335966
1,chr1-234,4.925634
2,chr2-283,4.577017
3,chr4-12,4.519867
4,chr1-467,4.454056


In [18]:
print(strata_df.shape)
strata_df.head()

(30474, 2)


,peak_id,peak_strata
0,chr9-2,35
1,chr1-234,35
2,chr2-283,50
3,chr4-12,40
4,chr1-467,40


### Download and Load motif matrices

In [19]:
%%bash
wget \
--user-agent="Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0" \
http://jaspar.genereg.net/download/CORE/JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt \
-O JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt

--2018-11-28 11:13:52--  http://jaspar.genereg.net/download/CORE/JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt
Resolving jaspar.genereg.net... 193.60.222.202
Connecting to jaspar.genereg.net|193.60.222.202|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214260 (209K) [text/plain]
Saving to: 'JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt'

     0K .......... .......... .......... .......... .......... 23%  178K 1s
    50K .......... .......... .......... .......... .......... 47%  360K 0s
   100K .......... .......... .......... .......... .......... 71% 9.77M 0s
   150K .......... .......... .......... .......... .......... 95% 5.98M 0s
   200K .........                                             100%  297K=0.5s

2018-11-28 11:13:53 (450 KB/s) - 'JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt' saved [214260/214260]



In [20]:
from meirlop import read_motif_matrices
known_motifs_filename = 'JASPAR2018_CORE_vertebrates_non-redundant_pfms_jaspar.txt'
known_motifs_file = open(known_motifs_filename, 'r')
motif_matrix_dict, motif_consensus_dict = read_motif_matrices(known_motifs_file)

## Scan for motifs
Create a dictionary, where the key is the motif id, and the value is a list of peaks containing the motif.

In [21]:
start = timer()
print(datetime.datetime.now())

from meirlop import format_scan_results, scan_motifs, get_background
scan_results_df, motif_peak_set_dict = format_scan_results(scan_motifs(motif_matrix_dict, 
                                                                       peak_sequence_dict, 
                                                                       bg = get_background(''.join(peak_sequence_dict.values())), 
                                                                       pval = 0.01, 
                                                                       pseudocount = 0.001, 
                                                                       window_size = 7))

end = timer()
runtime = end - start
print(f'{runtime} seconds')
print(datetime.datetime.now())

2018-11-28 11:13:54.156567
209.0934748277068 seconds
2018-11-28 11:17:23.250803


## Perform logistic regression analysis
Control for GC% as a covariate

In [22]:
covariates_df = dict_to_df(peak_gc_pct_dict, 'peak_id', 'peak_covariate')

In [23]:
print(covariates_df.shape)
covariates_df.head()

(30474, 2)


,peak_id,peak_covariate
0,chr15-1,37.437604
1,chr6-1,50.915141
2,chr1-1,43.926789
3,chr1-2,53.244592
4,chr12-1,45.757072


In [24]:
start = timer()
print(datetime.datetime.now())

from meirlop import analyze_peaks_with_lr
from tqdm import tqdm_notebook

lr_results_df = analyze_peaks_with_lr(peak_score_df = score_df,
                                      peak_set_dict = motif_peak_set_dict,
                                      peak_covariates_df = covariates_df,
                                      padj_method = 'fdr_bh',
                                      min_set_size = 1,
                                      max_set_size = np.inf,
                                      n_jobs = n_jobs,
                                      progress_wrapper = tqdm_notebook)

end = timer()
runtime = end - start
print(f'{runtime} seconds')
print(datetime.datetime.now())

2018-11-28 11:17:23.600520



117.57198852673173 seconds
2018-11-28 11:19:21.172768


In [25]:
lr_results_df.head(20)

,motif_id,coef,pval,padj,padj_sig,abs_coef
182,MA0517.1 STAT1::STAT2,0.226549,2.890335e-70,1.673504e-67,1,0.226549
31,MA0050.2 IRF1,0.163066,1.611660e-38,4.665755e-36,1,0.163066
375,MA0772.1 IRF7,0.152973,2.648969e-37,5.112511e-35,1,0.152973
577,MA1420.1 IRF5,0.119503,9.825213e-25,1.422200e-22,1,0.119503
69,MA0099.3 FOS::JUN,-0.111196,1.852787e-19,2.145527e-17,1,0.111196
547,MA1128.1 FOSL1::JUN,-0.107152,5.325775e-19,5.139373e-17,1,0.107152
563,MA1144.1 FOSL2::JUND,-0.100604,5.345893e-17,4.247259e-15,1,0.100604
554,MA1135.1 FOSB::JUNB,-0.100543,5.868406e-17,4.247259e-15,1,0.100543
557,MA1138.1 FOSL2::JUNB,-0.098077,3.471689e-16,1.827371e-14,1,0.098077
144,MA0478.1 FOSL2,-0.098056,1.020292e-16,6.563880e-15,1,0.098056


## Perform enrichment analysis with stratified permutations
We use an adaptation of GSEA prerank accounting for GC% in permutation testing.

In [27]:
start = timer()
print(datetime.datetime.now())

from meirlop import analyze_peaks_with_prerank
from tqdm import tqdm_notebook

rs = np.random.RandomState(1234)

analysis_results = analyze_peaks_with_prerank(peak_score_df = score_df, 
                                              peak_set_dict = motif_peak_set_dict, 
                                              peak_strata_df = strata_df, 
                                              min_set_size = 1, 
                                              max_set_size = np.inf, 
                                              nperm = 10, 
                                              nshuf = 100, 
                                              rs = rs, 
                                              n_jobs = n_jobs, 
                                              progress_wrapper = tqdm_notebook)

enrichment_score_results_df, shuffled_permuted_peak_data, peak_idx_to_peak_id = analysis_results

end = timer()
runtime = end - start
print(f'{runtime} seconds')
print(datetime.datetime.now())

2018-11-28 11:19:21.679088



2613.4009576682 seconds
2018-11-28 12:02:55.080225


In [28]:
enrichment_score_results_df.head(20)

,motif_id,es,nes,pval,n_more_extreme,fdr,n_all_null_nes_more_extreme,fdr_sig,abs_nes
0,MA0653.1 IRF9,0.597925,2.873886,0.000000,0,0.000000,0,1,2.873886
1,MA0652.1 IRF8,0.569183,2.837782,0.000000,0,0.000000,0,1,2.837782
2,MA1419.1 IRF4,0.570349,2.735087,0.000000,0,0.000000,0,1,2.735087
3,MA1418.1 IRF3,0.520757,2.687702,0.000000,0,0.000000,0,1,2.687702
4,MA0051.1 IRF2,0.509017,2.519631,0.000000,0,0.000000,0,1,2.519631
5,MA1420.1 IRF5,0.318710,1.799281,0.000000,0,0.000023,2,1,1.799281
6,MA0772.1 IRF7,0.323126,1.753466,0.000000,0,0.000057,5,1,1.753466
7,MA0517.1 STAT1::STAT2,0.290726,1.623906,0.000000,0,0.000621,54,1,1.623906
8,MA0050.2 IRF1,0.283426,1.535185,0.000000,0,0.001517,132,1,1.535185
9,MA0731.1 BCL6B,-0.590739,-1.464156,0.067395,37,0.000764,376,1,1.464156


In [29]:
enrichment_score_results_df[enrichment_score_results_df['fdr_sig'] == 1].shape

(216, 9)